In [1]:
import os
import time
from datetime import date
from dateutil.relativedelta import relativedelta
import re

from bs4 import BeautifulSoup
import pandas as pd
from selenium import webdriver
from selenium.common.exceptions import TimeoutException, ElementClickInterceptedException, WebDriverException
from selenium.webdriver import ActionChains
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.wait import WebDriverWait
from webdriver_manager.chrome import ChromeDriverManager


class Browser:
    def __init__(self):
        self.servico = Service(ChromeDriverManager().install())
        options = webdriver.ChromeOptions()
        options.headless = True
        self._driver = self.retornar_browser()

    def retornar_browser(self):
        return webdriver.Chrome(service=self.servico)

    def acessar_site(self, link):
        self._driver.get(link)

    def maximize_window(self):
        self._driver.maximize_window()

    def encontre_by_xpath(self, xpath):
        elemento = self._driver.find_element(By.XPATH, xpath)
        return elemento

    def encontre_elements_by_class(self, class_name):
        elementos = self._driver.find_elements(By.CLASS_NAME, class_name)
        return elementos

    @staticmethod
    def click_elemento(elemento):
        elemento.click()

    @staticmethod
    def send_keys(elemento, string):
        elemento.send_keys(string)

    def click_home_key(self):
        self._driver.find_element(By.XPATH, '//body').send_keys(Keys.HOME)

    def click_page_up_key(self):
        self._driver.find_element(By.XPATH, '//body').send_keys(Keys.PAGE_UP)

    def retorna_html_page(self):
        return self._driver.page_source

    def retornar_todas_as_abas_abertas(self):
        return self._driver.window_handles

    def mudar_aba_browser(self, numero_aba):
        abas = self.retornar_todas_as_abas_abertas()
        self._driver.switch_to.window(abas[numero_aba])

    @staticmethod
    def send_enter_key(elemento):
        elemento.send_keys(Keys.ENTER)

    def click_end_key(self):
        self._driver.find_element(By.XPATH, '//body').send_keys(Keys.END)

    def find_element_xpath_explicity_wait_visibility(self, tempo, xpath):
        elemento = WebDriverWait(self._driver, tempo).until(EC.visibility_of_element_located((By.XPATH, xpath)))
        return elemento

    def find_element_class_name_explicity_wait_visibility(self, tempo, class_name):
        elemento = WebDriverWait(self._driver, tempo).until(EC.visibility_of_element_located((By.CLASS_NAME, class_name)))
        return elemento

    def find_element_xpath_explicity_wait_clickable(self, tempo, xpath):
        # wait = WebDriverWait(self._driver, tempo)
        element = WebDriverWait(self._driver, tempo).until(EC.element_to_be_clickable((By.XPATH, xpath)))
        return element


# Clase que contém o itinerário de voos
class Itinerario:
    def __init__(self):
        self._lista_aeroporto = self.lista_aeroporto()

    # Retornar o itinerario a ser usado pelo browser
    def retorna_itinerario(self):
        for ida in self._lista_aeroporto:
            for volta in self._lista_aeroporto:
                if ida == volta:
                    pass
                else:
                    yield ida, volta

    # função que retornar a lista com os códigos dos aeroportos
    @staticmethod
    def lista_aeroporto():
        lista_aeroporto = [
            "SALVADOR",
            "FORTALEZA",
            "POA",
            "BELEM",
            "RECIFE",
            "CGR",
            "GIG",
            "GRU",
            "CWB",
            "BSB",
            "CNF",
            "VIX",
            "RBR",
            "SLZ",
            "ARACAJU",
            "GYN",
        ]
        return lista_aeroporto


# Classe que contém as ações no site da GOL
class Azul(Browser, Itinerario):
    def __init__(self):
        # self._driver = self.retornar_browser(servico)
        # Link utilizado para acessar site da Azul
        self._url = 'https://www.voeazul.com.br/br/pt/home'

        # Data atual
        self._data_atual = self.calcula_data_atual()
        self._data_ida = self.calcula_data_ida()
        self._data_volta = self.calcula_data_volta()

        self.data_atual_formatada = self._data_atual.strftime("%d/%m/%Y")
        self._data_ida_formatada = self._data_ida.strftime("%d/%m/%Y")
        self._data_volta_formatada = self._data_volta.strftime("%d/%m/%Y")

        self.coluna_nomes = pd.MultiIndex.from_tuples([('ORIGEM', f'Data Partida: {self._data_ida_formatada}'), ('DESTINO', f'Data Retorno: {self._data_volta_formatada}'), ('PREÇO DA PASSAGEM', f'Dia Coleta: {self.data_atual_formatada}')])
        # Df que guarda os itinerarios de origem
        self.df_ida_origem = pd.DataFrame(columns=self.coluna_nomes)

        # Df que guar os itinerários de volta
        self.df_volta_origem = pd.DataFrame(columns=self.coluna_nomes)

        # Df que guarda os itinerários de ida e volta
        self.df_preco_ida_volta = pd.DataFrame(columns=self.coluna_nomes)

        # Chamar super class Browser
        # super(Browser, self).__init__()

        # Chamar super class Itineario
        # super(Itinerario, self).__init__()
        Itinerario.__init__(self)

    # Método que realiza os processos do scraping
    def scraping(self):
        comeco = time.time()
        data_atual = self._data_atual.strftime("%d_%m_%Y")
        try:
            os.mkdir(f'Extração_AZUL_{data_atual}')
        except:
            pass
        for origem, destino in self.retorna_itinerario():
            while True:
                try:
                    print(origem)
                    print(destino)
                    # Para abrir o browser da automação
                    Browser.__init__(self)
                    # Acessar site da Azul
                    self.acessar_site(self._url)
                    self.maximize_window()
                    time.sleep(0.5)

                    xpath_input_origem = '//*[@id="Origem1"]'
                    elemento_input_origem = self.find_element_xpath_explicity_wait_clickable(10, xpath_input_origem)
                    self.click_elemento(elemento_input_origem)  # Para clicar no elemento de input de origem
                    time.sleep(0.55)
                    self.send_keys(elemento_input_origem, origem)  # Para escrever o aeroporto de origem
                    time.sleep(0.55)
                    self.send_enter_key(elemento_input_origem)

                    self._driver.find_element(By.XPATH, '//body').click()
                    xpath_input_destino = '//*[@id="Destino1"]'
                    elemento_input_destino = self.find_element_xpath_explicity_wait_visibility(10, xpath_input_destino)
                    time.sleep(0.55)
                    self.click_elemento(elemento_input_destino)  # Para clicar no elemento de input de destino
                    time.sleep(0.55)
                    self.send_keys(elemento_input_destino, destino)  # Para escrever o aeroporto de destino
                    time.sleep(0.55)
                    self.send_enter_key(elemento_input_destino)

                    xpath_label_partida = '//*[@id="spa-root"]/main/div/div[1]/div/div[1]/div[3]/div[1]/div[1]/div[1]/div[1]/div/nav/div/div/div[1]/div[1]/div[1]/ul/li/div/div[1]/div/div[2]/label'
                    elemento_label_partida = self.find_element_xpath_explicity_wait_clickable(5, xpath_label_partida)
                    self.click_elemento(elemento_label_partida)

                    xpath = '//*[@id="departure-1"]'
                    elemento_input_ida_data = self.find_element_xpath_explicity_wait_clickable(5, xpath)
                    data_ida = self._data_ida.strftime("%d%m%Y")
                    # data_ida = self._data_ida.strftime("25032023")
                    self.send_keys(elemento_input_ida_data, data_ida)

                    xpath_label_destino = '//*[@id="return-1"]'
                    elemento_label_destino = self.find_element_xpath_explicity_wait_clickable(5, xpath_label_destino)
                    data_volta = self._data_volta.strftime("%d%m%Y")
                    self.send_keys(elemento_label_destino, data_volta)

                    xpath = '//*[@id="spa-root"]/main/div/div[1]/div/div[1]/div[3]/div[1]/div[1]/div[1]/div[1]/div/nav/div/div/div[1]/div[1]/div[1]/ul/li/div/div[1]/div/div[2]/div/div/div[1]/button'
                    elemento_fechar_calendario = self.find_element_xpath_explicity_wait_clickable(5, xpath)
                    self.click_elemento(elemento_fechar_calendario)

                    self.click_home_key()
                    xpath = '//*[@id="spa-root"]/main/div/div[1]/div/div[1]/div[3]/div[1]/div[1]/div[1]/div[1]/div/nav/div/div/div[1]/div[1]/div[3]/div[3]/div/button'
                    elemento_but_presquisa = self.find_element_xpath_explicity_wait_clickable(5, xpath)
                    self.click_elemento(elemento_but_presquisa)

                    time.sleep(5)

                    # DataFrames que serão utilizados caso não haja passagens
                    df_sem_passagem_ida = self.cria_df_sem_passagem(origem, destino)
                    df_sem_passagem_volta = self.cria_df_sem_passagem(destino, origem)

                    # Verificar se há passagens para essa rota
                    xpath = '//*[@id="spa-root"]/main/div/div[1]/div/div[1]/div[3]/div[1]/div[1]/div[1]/div/div[1]/section[2]/div/div/div/section/div[1]'
                    try:
                        primeiro_voo_ida = self.find_element_xpath_explicity_wait_visibility(15, xpath)
                        print('Há voos para essa rota')
                        self.click_end_key()
                        time.sleep(0.3)
                        self.click_page_up_key()
                        time.sleep(0.3)

                        # Laço para clicar em exibir mais voos
                        while True:
                            try:
                                xpath = '//*[@id="load-more-button"]'
                                tempo = 5
                                self.click_end_key()
                                time.sleep(1.23)
                                self.click_page_up_key()
                                time.sleep(1.23)
                                elemento_carregar_mais_passagens = self.find_element_xpath_explicity_wait_visibility(tempo, xpath)
                                # actions = ActionChains(self._driver)
                                # actions.move_to_element(elemento_carregar_mais_passagens).perform()
                                self.click_elemento(elemento_carregar_mais_passagens)
                                time.sleep(1.3)
                            except TimeoutException:
                                break

                        price_final_ida = self.realizar_extracao_precos(0)
                        for i in range(len(price_final_ida)):
                            df_ida_juntar = pd.DataFrame(data=[origem, destino, price_final_ida[i]]).T
                            df_ida_juntar.columns = self.coluna_nomes
                            self.df_preco_ida_volta = pd.concat([self.df_preco_ida_volta, df_ida_juntar], axis=0)
                            self.df_ida_origem = pd.concat([self.df_ida_origem, df_ida_juntar], axis=0)

                        print(price_final_ida)
                        self.click_home_key()
                        time.sleep(0.55)
                        self.click_elemento(primeiro_voo_ida)

                        # Clicar em pesquisar o voo de volta
                        xpath = '//*[@id="spa-root"]/main/div/div[1]/div/div[1]/div[3]/div[1]/div[1]/div[1]/div/div[1]/section[2]/div/div/div/section/div/div[2]/div/ul/li[2]/ul/li/button'
                        element_but_pesquisa_volta = self.find_element_xpath_explicity_wait_visibility(5, xpath)
                        self.click_elemento(element_but_pesquisa_volta)
                        self.click_home_key()

                        xpath = '//*[@id="spa-root"]/main/div/div[1]/div/div[1]/div[3]/div[1]/div[1]/div[1]/div/div[1]/section[2]/div/div[2]/div/section/div'
                        try:
                            primeiro_voo_retorno = self.find_element_xpath_explicity_wait_visibility(5, xpath)
                            print('Há voos para de volta essa rota')

                            # Laço para clicar em exibir mais voos
                            while True:
                                try:
                                    # self.click_end_key()
                                    xpath = '//*[@id="load-more-button"]'
                                    time.sleep(2.23)
                                    self.click_end_key()
                                    time.sleep(1.23)
                                    # self.click_page_up_key()
                                    # time.sleep(1.23)
                                    elemento_but_mais_voos = self.find_element_xpath_explicity_wait_visibility(3, xpath)
                                    self.click_elemento(elemento_but_mais_voos)
                                    time.sleep(1.23)
                                except TimeoutException:
                                    break

                            price_final_volta = self.realizar_extracao_precos(1)
                            print(price_final_volta)
                            for i in range(len(price_final_volta)):
                                df_ida_juntar = pd.DataFrame(data=[destino, origem, price_final_volta[i]]).T
                                df_ida_juntar.columns = self.coluna_nomes
                                self.df_preco_ida_volta = pd.concat([self.df_preco_ida_volta, df_ida_juntar], axis=0)
                                self.df_volta_origem = pd.concat([self.df_volta_origem, df_ida_juntar], axis=0)

                        except TimeoutException:
                            print('Não há voos de volta!')
                            self.df_preco_ida_volta = pd.concat([self.df_preco_ida_volta, df_sem_passagem_volta], axis=0)
                            self.df_volta_origem = pd.concat([self.df_volta_origem, df_sem_passagem_volta], axis=0)

                    except TimeoutException:
                        print('Não há voos para essa rota')
                        self.df_preco_ida_volta = pd.concat([self.df_preco_ida_volta, df_sem_passagem_ida], axis=0)
                        self.df_ida_origem = pd.concat([self.df_ida_origem, df_sem_passagem_ida], axis=0)
                        self.df_preco_ida_volta = pd.concat([self.df_preco_ida_volta, df_sem_passagem_volta], axis=0)
                        self.df_volta_origem = pd.concat([self.df_ida_origem, df_sem_passagem_volta], axis=0)

                    print(self.df_preco_ida_volta)
                    print(self.df_ida_origem)
                    print(self.df_volta_origem)
                    self._driver.quit()
                    break

                except TimeoutException:
                    self._driver.quit()
                    print('Deu ruim, vai repetir de novo!')

                except ElementClickInterceptedException:
                    self._driver.quit()
                    print('Deu ruim, vai repetir de novo!')

                except WebDriverException:
                    print('Erro: WebDriverException, vai repetir de novo!')
                    self._driver.quit()

            self.df_preco_ida_volta.to_excel(f'Extração_AZUL_{data_atual}/df_preco_azul_ida_volta_{data_atual}.xlsx')
            self.df_ida_origem.to_excel(f'Extração_AZUL_{data_atual}/df_preco_azul_ida_origem_{data_atual}.xlsx')
            self.df_volta_origem.to_excel(f'Extração_AZUL_{data_atual}/df_preco_azul_volta_destino_{data_atual}.xlsx')
        final = time.time()
        print(final - comeco)65

    def cria_df_sem_passagem(self, aero1, aero2):
        data_sem_passagem = 'Sem voos disponíveis'
        df_sem_passagem = pd.DataFrame(data=[aero1, aero2, data_sem_passagem]).T
        df_sem_passagem.columns = self.coluna_nomes
        return df_sem_passagem

    def realizar_extracao_precos(self, numero_dado):
        html = self.retorna_html_page()
        soup = BeautifulSoup(html, 'html.parser')
        price = soup.findAll('div', class_='trip-container')
        soup = BeautifulSoup(str(price[numero_dado]), 'html.parser')
        precos = soup.find_all('h4')
        lts_preco = []
        for preco in precos:
            texto = preco.get_text()
            match = re.search('(R\\$)(\\d{1,3})?(.)?(\\d{1,3})(,)(\\d{1,2})', texto)
            if match:
                lts_preco.append(match.group())
        tamanho = len(lts_preco)
        price_final = [lts_preco[i] for i in range(tamanho) if i % 3 == 0]
        return price_final

    @staticmethod
    def calcula_data_atual():
        return date.today()

    def calcula_data_ida(self):
        data_ida = self._data_atual + relativedelta(months=2)
        return data_ida

    def calcula_data_volta(self):
        data_volta = self._data_ida + relativedelta(days=8)
        return data_volta


def scraping_azul():
    objeto_azul = Azul()
    objeto_azul.scraping()


if __name__ == '__main__':
    scraping_azul()

SALVADOR
FORTALEZA
Há voos para essa rota
['R$739,36', 'R$803,36', 'R$803,36', 'R$803,36', 'R$803,36', 'R$1.023,36', 'R$803,36', 'R$858,36', 'R$803,36', 'R$803,36', 'R$803,36', 'R$803,36']
Há voos para de volta essa rota
['R$567,61', 'R$567,61', 'R$913,61', 'R$624,61', 'R$567,61', 'R$567,61', 'R$567,61', 'R$567,61', 'R$567,61', 'R$2.152,61', 'R$2.152,61', 'R$749,61', 'R$913,61', 'R$624,61', 'R$567,61', 'R$567,61', 'R$567,61', 'R$567,61', 'R$567,61', 'R$567,61']
                    ORIGEM                  DESTINO      PREÇO DA PASSAGEM
  Data Partida: 09/04/2023 Data Retorno: 17/04/2023 Dia Coleta: 09/02/2023
0                 SALVADOR                FORTALEZA               R$739,36
0                 SALVADOR                FORTALEZA               R$803,36
0                 SALVADOR                FORTALEZA               R$803,36
0                 SALVADOR                FORTALEZA               R$803,36
0                 SALVADOR                FORTALEZA               R$803,36
0        

[WDM] - Downloading: 100%|██████████| 6.79M/6.79M [00:00<00:00, 25.4MB/s]


Há voos para essa rota
['R$1.093,73', 'R$1.093,73', 'R$1.093,73', 'R$1.093,73', 'R$1.093,73', 'R$1.093,73', 'R$1.093,73', 'R$1.093,73', 'R$1.192,73', 'R$1.192,73', 'R$1.192,73', 'R$1.192,73', 'R$1.192,73', 'R$1.192,73', 'R$1.192,73', 'R$1.429,73', 'R$3.093,73', 'R$3.093,73', 'R$1.192,73', 'R$1.192,73']
Há voos para de volta essa rota
['R$1.181,42', 'R$1.181,42', 'R$1.181,42', 'R$1.181,42', 'R$1.880,42', 'R$2.482,42', 'R$1.418,42', 'R$2.482,42', 'R$1.718,42', 'R$2.482,42', 'R$1.880,42', 'R$1.718,42', 'R$1.082,42', 'R$898,42', 'R$1.082,42', 'R$898,42', 'R$1.082,42', 'R$898,42', 'R$898,42', 'R$898,42']
                     ORIGEM                  DESTINO      PREÇO DA PASSAGEM
   Data Partida: 09/04/2023 Data Retorno: 17/04/2023 Dia Coleta: 09/02/2023
0                  SALVADOR                FORTALEZA               R$739,36
0                  SALVADOR                FORTALEZA               R$803,36
0                  SALVADOR                FORTALEZA               R$803,36
0            